# Introduction to Mixed-Effects Models

## What are Mixed-Effects Models?

Mixed-effects models, also known as hierarchical or multilevel models, are statistical models that incorporate both fixed and random effects. These models are particularly useful when dealing with data that has a hierarchical structure, such as repeated measures from the same subjects, nested data, or data collected from multiple groups.


### Key Concepts

1. **Fixed Effects**:
   - Fixed effects represent the average effect of predictors (independent variables) across all observations. These are the traditional coefficients estimated in standard regression models.

2. **Random Effects**:
   - Random effects account for variability across groups or subjects. They allow the model to capture individual differences by including random intercepts or slopes, enabling a more flexible fit to the data.


### Why Use Mixed-Effects Models?

- **Hierarchical Data**: When data are collected in groups (e.g., students within schools, patients within clinics), mixed-effects models appropriately account for the dependence of observations within the same group.
- **Repeated Measures**: They are ideal for analyzing longitudinal data, where the same subjects are measured multiple times.
- **Increased Precision**: By modeling random effects, mixed-effects models can provide more accurate estimates of fixed effects by borrowing strength across groups.


### Model Specification

A typical mixed-effects model can be represented as:

$$
Y_{ij} = \beta_0 + \beta_1 X_{ij} + u_{0j} + \epsilon_{ij}
$$

Where:
- $Y_{ij}$ is the response variable for the $i$-th observation in the $j$-th group.
- $\beta_0$ is the fixed intercept.
- $\beta_1$ is the fixed effect for the predictor $X$.
- $u_{0j}$ is the random effect for the $j$-th group (random intercept).
- $\epsilon_{ij}$ is the residual error.


### Implementation in Python

In Python, mixed-effects models can be fitted using the `statsmodels` library, which provides the `MixedLM` function. This allows researchers to specify both fixed and random effects in a straightforward manner.


### About the notebook

In this notebook I am trying to demonstrate how we fit a mixed effect model and evaluate its performance.  
We will fit a mixed effect model for a simple dataset which shows the reaction time of the individuals for 10 days during when they were sleep deprived.

The objective of this notebook is to use this dataset and demonstrate the methods of fitting a mixed effect model for a longitudinal data.


## Importing libraries and Data

In [5]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

In [6]:
data = pd.read_csv("sleepstudy.csv")

In [7]:
data.head()

,Unnamed: 0,Reaction,Days,Subject
0,1,249.5600,0,308
1,2,258.7047,1,308
2,3,250.8006,2,308
3,4,321.4398,3,308
4,5,356.8519,4,308


In [8]:
data['Subject'].nunique()

18

In [9]:
data['Days'].nunique()

10

The study is conducted in 18 subjects for 10 days.

## Model specifications

In this dataset we can assume the fixed and random effects as below

1. The effect of sleep deprivation will have a fixed effect on every subjects.
2. Each subject can have a different baseline reaction time compared to others, so we can have a random effect on the baseline reaction time for each individual

In [15]:
# Set random seed for reproducibility
np.random.seed(123)

model = smf.mixedlm("Reaction ~ Days", data, groups=data["Subject"])


In [16]:
# Fit the model
fit = model.fit()

# Output the summary of the model fit
print(fit.summary())

         Mixed Linear Model Regression Results
Model:             MixedLM Dependent Variable: Reaction 
No. Observations:  180     Method:             REML     
No. Groups:        18      Scale:              960.4568 
Min. group size:   10      Log-Likelihood:     -893.2325
Max. group size:   10      Converged:          Yes      
Mean group size:   10.0                                 
--------------------------------------------------------
           Coef.   Std.Err.   z    P>|z|  [0.025  0.975]
--------------------------------------------------------
Intercept  251.405    9.747 25.794 0.000 232.302 270.508
Days        10.467    0.804 13.015 0.000   8.891  12.044
Group Var 1378.176   17.156                             



Coefficient values as below:

1. Intercept = 251.405    
2. Days  (Fixed Effect)  =   10.467  
3. Group Var (Random effect) = 1378.176 

## Model Evaluation

In [26]:

# Predict values using the mixed-effects model
predictions = fit.predict()  #

# Actual values (observed)
actual_values = data['Reaction']  

# Calculate RMSE
rmse = np.sqrt(np.mean((predictions - actual_values) ** 2))
print("Root Mean Squared Error (RMSE):", rmse)

# Calculate R-squared
ss_total = np.sum((actual_values - np.mean(actual_values)) ** 2)  # Total sum of squares
ss_residual = np.sum((actual_values - predictions) ** 2)  # Residual sum of squares
r_squared = 1 - (ss_residual / ss_total)
print("R-squared:", r_squared)

# Calculate MAPE
mape = np.mean(np.abs((actual_values - predictions) / actual_values)) * 100
print("Mean Absolute Percentage Error (MAPE):", mape)

# Calculate accuracy (for regression, typically based on a threshold, e.g., ±10% error)
accuracy = np.mean(np.abs((predictions - actual_values) / actual_values) < 0.1) * 100  # 10% threshold
print("Accuracy (within 10%):", round(accuracy,2), "%")


Root Mean Squared Error (RMSE): 47.448897509757494
R-squared: 0.28647139510771014
Mean Absolute Percentage Error (MAPE): 12.674582143277814
Accuracy (within 10%): 53.89 %




Above metrics are general evaluation metrics used for linear models.  
These metrics values indicating out model is moderately good one. (Accuracy = 53.89% and MAPE is 12.6 %)

As we are building fixed effect models its crucial to look at the models evaluation by cosidering the random effect components as well.
Lets explore some of most common evaluation of Mixed effect models.

**a. Coefficient Significance (P-values)**

We can check the p-values of coefficients to see if they are significant or not.   
In the summary both p-values are less than 0.05 **so both coefficients are significant.**

**b. Confidence Intervals**

Confidence intervals help us understand the precision of the fixed effects estimates.

In [17]:
fit.conf_int()

,0,1
Intercept,232.301907,270.508303
Days,8.891041,12.043531
Group Var,0.349942,2.519892


In general Narrower intervals indicate more precise estimates, while wider intervals suggest more uncertainty.  
For both fixed and random coefficients we can see the values are narrower, so we can say the uncertainity is very less.

**c. Intra-Class Correlation (ICC)**

ICC measures the proportion of total variance that is attributable to the random effects (between-group variability). It tells you how much of the variation in the outcome (e.g., math score) is due to differences between groups (schools) rather than individual-level variability.

The formula for ICC is:

$$
ICC = \frac{\sigma_{\text{random}}^2}{\sigma_{\text{random}}^2 + \sigma_{\text{residual}}^2}
$$

Where:
- $\sigma_{\text{random}}^2$ is the variance of the random effects (e.g., between schools),
- $\sigma_{\text{residual}}^2$ is the residual (within-group) variance.


In [20]:
# Extract the variance components
random_effect_variance = fit.cov_re.iloc[0, 0]  # Group variance
residual_variance = fit.scale  # Residual variance

# Calculate ICC
icc = random_effect_variance / (random_effect_variance + residual_variance)
print("Intra-Class Correlation (ICC):", icc)


Intra-Class Correlation (ICC): 0.5893084022885959


**ICC of 0.589 indicates moderate to good agreement among groups (Here, within individuals). This suggests that a significant portion of the total variance of reaction time is attributable to the grouping factor(individual random effect factor).**

**d. Likelihood Ratio Test (LRT)**

A likelihood ratio test is used to compare a mixed-effects model with simpler (nested) models. This can help determine if adding random effects significantly improves model fit.


Null Hypothesis: The null hypothesis is that the simpler model (in our case, the OLS model without random effects) is adequate to explain the data.

In [24]:
from statsmodels.base.model import LikelihoodModelResults
from scipy import stats

# Fit a simpler model (without random effects)
simpler_model = smf.ols("Reaction ~ Days", data).fit()

# Likelihood ratio test
lr_stat = -2 * (simpler_model.llf - fit.llf)  # Compare log-likelihoods
lr_p_value = 1 - stats.chi2.cdf(lr_stat, df=1)  # df=1, as we compare one additional random effect
print("LRT P-value:", lr_p_value)

LRT P-value: 0.0


The P-Value is 0.0 which is lesss than 0.05, so we can reject null hypothesis and confirm that adding a random effect is helping.

## Conclusion

In this analysis, we employed a **fixed-effects model** to investigate the impact of sleep deprivation on reaction times in a longitudinal dataset, capturing changes over 10 days. This methodology is particularly well-suited for analyzing repeated measurements from the same subjects, as it effectively controls for unobserved heterogeneity.

The fixed-effects modeling approach offers a robust framework for analyzing longitudinal data, particularly in behavioral studies like sleep deprivation. 

This analysis serves as a demonstration of the effectiveness of fixed-effects modeling in longitudinal studies and its evaluation techniques.
